# Module 1: PyTorch Tensors & Fundamentals

**Level:** Beginner  
**Duration:** ~2 hours  
**Real-World Use Case:** Understanding how numerical data flows through deep learning pipelines

---

## The Big Picture — Two Analogies

### 🧠 Human Brain Analogy
Your brain processes reality by converting everything — sight, sound, touch — into **electrical signals**: streams of numbers firing across billions of neurons. PyTorch tensors are the same idea: they are the universal language in which a computer encodes *all* information before a neural network can think about it.

- A **photo** becomes a 3D tensor of pixel brightness values `(3, 224, 224)` — like the retina converting light into nerve signals.
- A **sentence** becomes a 2D tensor of word embeddings `(seq_len, 768)` — like the auditory cortex encoding sound into meaning.
- The **network's weights** are tensors too — like the synaptic strengths between neurons, updated every time the network learns.

> *Tensors are to PyTorch what neurons are to the brain — the fundamental unit everything is built from.*

### ⚙️ Engineer Analogy
Think of a tensor as a **typed, multi-dimensional buffer in memory** — like a structured data array in a CAD system or a register file in a CPU. Before any computation can happen, an engineer must:
1. **Allocate** the buffer (create the tensor)
2. **Specify its format** (dtype, shape)
3. **Place it on the right hardware** (CPU vs GPU)

This module teaches you to do exactly that.

---

## What You'll Learn
- Creating and manipulating tensors (the core data structure in PyTorch)
- Tensor operations: math, indexing, reshaping
- Moving tensors to GPU
- Autograd: automatic differentiation
- Connecting PyTorch tensors to NumPy

## Mental Model
Think of a **Tensor** as a multi-dimensional array (like NumPy ndarray) that:
1. Can run on GPU for massive speed gains
2. Tracks computation history for automatic gradient calculation (backprop)

```
Scalar  → 0D tensor  → single number:   42             (one neuron's output)
Vector  → 1D tensor  → list:            [1, 2, 3]      (one layer's activations)
Matrix  → 2D tensor  → grid:            [[1,2],[3,4]]  (a weight layer W)
Tensor  → 3D+        → e.g. RGB image:  (3, 224, 224)  (what the eye sends in)
```

In [ ]:
# ── 1.0  Environment Check ────────────────────────────────────────────────
# 🧠 Brain: Before your brain does any thinking, it needs blood flow (energy) and
#           the right neurotransmitters ready. This cell is our "brain warm-up" —
#           confirming the tools (PyTorch, NumPy) and hardware (CPU/GPU) are alive.
#
# ⚙️ Engineer: Like checking power rails and firmware version before running
#              a microcontroller. No point writing code if the runtime isn't ready.

import torch
import numpy as np
import matplotlib.pyplot as plt

print(f"PyTorch version : {torch.__version__}")
print(f"CUDA available  : {torch.cuda.is_available()}")   # GPU = turbo mode
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device    : {device}")

## 1.1  Creating Tensors — Six Ways

### 🧠 Brain Analogy
Before your brain can reason about something, it must **encode** it — convert the raw world into a form neurons can process. When you see the number "3", your visual cortex fires a specific pattern of neurons; that pattern *is* the brain's tensor.

Creating a tensor in PyTorch is like **choosing how to encode information** before passing it into a network:
- **From a list** → like a person recalling a memory and writing it down precisely
- **Zeros / Ones** → like a blank or fully-activated neuron layer (initial state before learning)
- **Random (randn)** → like the random initial synaptic weights when a baby is born — noisy, but ready to be shaped by experience
- **From NumPy** → like translating a document between two languages that share memory (no copying needed!)

### ⚙️ Engineer Analogy
Think of each creation method as a different **memory allocation strategy** for a typed buffer:

| Method | Engineering Equivalent |
|--------|------------------------|
| `torch.tensor(list)` | Initialise buffer from known values (hard-coded data) |
| `torch.zeros / ones` | `calloc` — allocate and zero-fill; safe known initial state |
| `torch.rand / randn` | Random seed initialisation — like Xavier/He weight init in practice |
| `torch.arange` | Loop counter array — evenly-spaced index buffer |
| `torch.linspace` | Uniform sample grid — like frequency bins in an FFT |
| `torch.from_numpy` | Shared-memory view — **zero-copy** pointer aliasing (changes in one reflect in the other!) |

> **Key engineering insight:** `torch.from_numpy` does NOT copy data. Both the NumPy array and the tensor point to the same memory block. Modify one, and the other changes too — like two pointers aliasing the same address.

In [ ]:
# ── From Python lists ────────────────────────────────────────────────────
# 🧠 Like manually writing down a memory: "I saw 1, 2, 3, 4"
# ⚙️ Like hard-coding known register values at boot time
t_from_list = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
print("From list:\n", t_from_list)

# ── Zeros / Ones / Random ────────────────────────────────────────────────
# 🧠 zeros  → a silent brain region (no activation yet)
# 🧠 ones   → a fully saturated/active region
# 🧠 rand   → random background neural noise (uniform)
# 🧠 randn  → natural neural noise — most activity near 0, rare extremes (bell curve)
# ⚙️ zeros/ones → calloc vs memset(1) — deterministic initial state
# ⚙️ randn  → the Xavier/He initialisation engineers use to prevent vanishing gradients
zeros  = torch.zeros(3, 4)                    # 3×4 filled with 0s
ones   = torch.ones(3, 4)                     # 3×4 filled with 1s
rand   = torch.rand(3, 4)                     # uniform [0,1)
randn  = torch.randn(3, 4)                    # standard normal N(0,1)

# ── Evenly-spaced sequences ──────────────────────────────────────────────
# 🧠 Like a metronome ticking at fixed intervals — predictable, structured time steps
# ⚙️ arange → C-style loop index  |  linspace → sample grid (like FFT frequency bins)
arange = torch.arange(0, 10, step=2)          # like np.arange:   [0, 2, 4, 6, 8]
linsp  = torch.linspace(0, 1, steps=5)        # like np.linspace: [0, .25, .5, .75, 1]

print("\nzeros:\n",  zeros)
print("arange:",    arange)
print("linspace:",  linsp)

# ── From NumPy — ZERO-COPY shared memory ─────────────────────────────────
# 🧠 Like two parts of the brain accessing the same long-term memory without
#    duplicating it — efficient and always in sync.
# ⚙️ Pointer aliasing: np_arr and t_np share the same RAM block.
#    WARNING: modifying t_np will also mutate np_arr!
np_arr  = np.array([1.0, 2.0, 3.0])
t_np    = torch.from_numpy(np_arr)            # shares memory — no copy!
print("\nFrom NumPy:", t_np)

# Prove they share memory:
np_arr[0] = 99.0
print("After mutating np_arr[0]=99, t_np:", t_np)  # t_np[0] also becomes 99

## 1.2  Tensor Attributes

Every tensor carries three key properties: **shape**, **dtype**, **device**.

### 🧠 Brain Analogy
Think of a tensor as a group of neurons that carries metadata about itself:
- **`shape`** — *how many neurons are in the group, and how they're arranged* (e.g. a visual cortex column is arranged in layers — 4 batches × 3 channels × 2 features)
- **`dtype`** — *the precision of each neuron's signal* — float32 is like a standard nerve signal; int8 is a coarser, compressed approximation (like how your peripheral vision is lower resolution than your fovea)
- **`device`** — *which part of the brain (or body) is doing the processing* — CPU is the deliberate prefrontal cortex; GPU is the fast, parallel cerebellum

### ⚙️ Engineer Analogy
A tensor's metadata is its **hardware spec sheet**:
- **`shape`** → array dimensions = how the buffer is laid out in memory (row-major, like a 3D grid)
- **`dtype`** → data type = `float32` costs 4 bytes/element, `int8` costs 1 byte — choose wisely for speed vs precision tradeoffs (quantisation!)
- **`device`** → compute unit = CPU (general-purpose, low-latency) vs GPU (high-throughput, high-latency per op but massively parallel)
- **`numel()`** → total allocation size = `shape[0] × shape[1] × ... × shape[n]` — tells you the memory footprint

> **Pro tip:** In production ML, engineers spend a lot of time reducing `dtype` (e.g. float32 → float16 or int8) to fit more data into GPU memory without losing accuracy. This is called **quantisation**.

In [ ]:
# 🧠 Imagine a 3D brain region: 4 people (batch), each with 3 sensory channels,
#    each channel producing 2 feature values. That's x.shape = (4, 3, 2).
# ⚙️ A 3-rank tensor of shape (4,3,2) — 24 float32 values = 24 × 4 = 96 bytes in memory.
x = torch.randn(4, 3, 2)   # 4 batches, 3 channels, 2 features

print("Shape  :", x.shape)          # torch.Size([4, 3, 2])  ← the memory layout
print("ndim   :", x.ndim)           # 3                      ← rank of the tensor
print("dtype  :", x.dtype)          # torch.float32          ← 4 bytes per element
print("device :", x.device)         # cpu                    ← where it lives
print("numel  :", x.numel())        # 4*3*2 = 24 elements total (= bytes/4 for float32)

# ── Changing dtype: trading precision for speed/memory ───────────────────
# 🧠 Like switching from HD to SD vision — less detail, but faster to process
# ⚙️ float32 → int32: drops fractional precision; saves no memory here (both 4 bytes),
#    but int8 (not shown) would cut to 1 byte — used in quantised inference engines
x_int = x.to(torch.int32)
print("\nAs int32 dtype:", x_int.dtype)

## 1.3  Essential Tensor Operations

### 🧠 Brain Analogy
When neurons communicate, they don't just pass raw signals — they **combine, scale, and summarise** them:
- **Element-wise ops** (`a + b`, `a * b`) → like two neurons that both sense the same stimulus and their signals are simply added or multiplied together at the next neuron (synaptic summation)
- **Matrix multiplication** (`a @ b`) → like a **full layer of neurons**: every input neuron talks to every output neuron, each connection weighted differently. This is *exactly* what a dense (linear) layer does: `output = W @ input`
- **Aggregation** (`sum`, `mean`, `max`) → like a region of the brain **summarising** its inputs into a single decision signal — e.g. the basal ganglia voting "go" or "no-go" based on the overall activity level

### ⚙️ Engineer Analogy
- **Element-wise ops** → SIMD (Single Instruction, Multiple Data) vector operations — the CPU/GPU applies one instruction to all 32/64/128 elements simultaneously
- **`a @ b` (MatMul)** → the most important op in deep learning; on a GPU it maps to **GEMM** (General Matrix-Matrix Multiply), optimised using BLAS libraries (cuBLAS on CUDA). This single operation accounts for ~90% of compute in transformer models
- **`argmax`** → like a priority encoder in digital logic — outputs the *index* of the winning signal
- **`dim=0` vs `dim=1`** → reducing along a dimension collapses that axis; think of it as collapsing rows (dim=0) or columns (dim=1) of a spreadsheet

> **Why dim matters:** `sum(dim=0)` sums across rows (result has shape of one row), `sum(dim=1)` sums across columns (result has shape of one column). Getting this wrong is one of the most common PyTorch bugs.

In [ ]:
a = torch.tensor([[1., 2.], [3., 4.]])
b = torch.tensor([[5., 6.], [7., 8.]])

# ── Element-wise arithmetic ──────────────────────────────────────────────
# 🧠 Two neuron groups firing simultaneously — signals combine position-by-position
# ⚙️ SIMD ops: each element processed in parallel, same instruction applied to all
print("a + b:\n",  a + b)          # same as torch.add(a, b)
print("a * b:\n",  a * b)          # element-wise multiply (NOT matrix multiply!)
print("a ** 2:\n", a ** 2)         # squaring each activation — like a power amplifier

# ── Matrix multiplication ────────────────────────────────────────────────
# 🧠 THE core brain operation: every input neuron connects to every output neuron.
#    a is the input activations (2×2), b is the weight matrix (2×2).
#    Result[i,j] = dot product of row i of a with col j of b.
# ⚙️ Maps to GEMM on GPU — the single most-optimised operation in deep learning.
#    In a transformer: Q @ K.T is this operation, done billions of times per second.
print("\nMatMul a @ b:\n", a @ b)  # same as torch.matmul(a, b)

# ── Aggregation ──────────────────────────────────────────────────────────
# 🧠 The brain compressing many signals into one summary statistic to make a decision
# ⚙️ Reduction operations — collapse the tensor to a scalar or lower-rank tensor
print("\nSum:",     a.sum())        # total energy across all neurons
print("Mean:",    a.mean())        # average activation level
print("Max: ",    a.max())         # the most active neuron's value
print("Argmax:",  a.argmax())      # WHICH neuron was most active (its flat index)
                                   # argmax([1,2,3,4]) → 3  (index of 4)

# ── Axis-wise aggregation ────────────────────────────────────────────────
# 🧠 Summarising along a specific brain axis:
#    dim=0 → "what was the average signal across the batch of inputs?" (column summary)
#    dim=1 → "what was the total signal for each individual input?" (row summary)
# ⚙️ Think of it as collapsing a spreadsheet:
#    dim=0 → collapse rows → result shape = (num_cols,)
#    dim=1 → collapse cols → result shape = (num_rows,)
print("\nSum along rows (dim=0):", a.sum(dim=0))  # [1+3, 2+4] = [4, 6]  (column sums)
print("Sum along cols (dim=1):", a.sum(dim=1))   # [1+2, 3+4] = [3, 7]  (row sums)

## 1.4  Indexing, Slicing & Reshaping

### 🧠 Brain Analogy
Your brain doesn't attend to everything at once — it uses **selective attention** to focus on what matters:
- **Indexing** (`x[1, 2]`) → like your brain spotlight-focusing on a single pixel in your visual field — everything else is still there, but ignored
- **Slicing** (`x[:2]`, `x[:, 2]`) → like peripheral vision — you're processing a *region* of the scene, not a single point
- **Reshape / View** → like rearranging the furniture in a room without changing what's in it. The data stays the same; only the *layout* changes. Your brain does this when it re-contextualises information (e.g. a 3D object mentally rotated)
- **unsqueeze / squeeze** → like adding a "batch dimension" to a single thought — your brain can simultaneously hold "what is this?" (a single example) and "how does this compare across many examples?" (a batch)

### ⚙️ Engineer Analogy
- **Indexing** → pointer arithmetic into a contiguous memory buffer: `x[i, j]` = `base_ptr + i * stride_0 + j * stride_1`
- **Slicing** → returns a **view** (not a copy!) — like a window into the same memory block with different start/stride
- **`reshape` vs `view`** → both reinterpret the same memory layout. `view` requires contiguous memory; `reshape` will copy if needed — similar to `reinterpret_cast` vs `memcpy` + cast
- **`unsqueeze(dim)`** → inserts a size-1 dimension — critical for **broadcasting** (next section). Used constantly to align tensor shapes for batch operations: `(3,)` → `(1, 3)` makes it broadcastable over a batch
- **`squeeze()`** → removes all size-1 dimensions — like stripping redundant wrapper layers

> **Memory insight:** Slicing and `view` create tensors that **share memory** with the original. Modifying the slice modifies the source. Call `.clone()` if you need an independent copy.

In [ ]:
# Build a 3×4 matrix from a range — like a 3-row spreadsheet with 4 columns
# 🧠 Imagine 3 people, each described by 4 brain measurements
# ⚙️ arange(1,13) creates [1..12]; reshape reinterprets the flat buffer as 3×4 grid
x = torch.arange(1, 13).reshape(3, 4)   # shape (3,4)
print("Original:\n", x)

# ── Indexing — spotlight attention ──────────────────────────────────────
# 🧠 Zoom in on exactly one neuron's reading
# ⚙️ Resolves to: base + 0*stride_0 = base (first row of the buffer)
print("\nRow 0      :", x[0])             # first row    → [1, 2, 3, 4]
print("Col 2      :", x[:, 2])           # entire 3rd column → [3, 7, 11]
print("x[1,2]     :", x[1, 2])           # single element: row 1, col 2 → 7
print("Rows 0-1   :\n", x[:2])           # first two rows (slice = view, not copy!)

# ── Reshape / View — reorganise without copying ───────────────────────────
# 🧠 Same 12 "facts", rearranged into a different mental grid
# ⚙️ Both return views sharing the same memory — just different stride metadata
print("\nReshaped to (4,3):\n", x.reshape(4, 3))  # 4 rows, 3 cols — same 12 values
print("\nFlattened      :", x.flatten())           # collapse to 1D: [1,2,...,12]
print("\nView as (2,6)  :\n", x.view(2, 6))        # view requires contiguous storage

# ── unsqueeze / squeeze — adding/removing phantom dimensions ──────────────
# 🧠 unsqueeze: "treat this as a batch of 1" — same data, wider context
# ⚙️ Used to align shapes for broadcasting or for batch-first APIs that expect (N, ...)
y = torch.tensor([1., 2., 3.])    # shape (3,)  — a single vector
print("\ny.unsqueeze(0) shape:", y.unsqueeze(0).shape)  # (1,3) ← row vector
print("y.unsqueeze(1) shape:", y.unsqueeze(1).shape)   # (3,1) ← column vector

# 🧠 squeeze: "stop pretending this is a batch" — strip the dummy dimension
# ⚙️ Removes all dims of size 1 — (1,3,1) → (3,)
z = torch.zeros(1, 3, 1)
print("squeeze shape:", z.squeeze().shape)             # (3,)

## 1.5  Broadcasting

Broadcasting lets tensors of different shapes work together — similar to NumPy.

```
Shape (3,1) + Shape (1,4) → Shape (3,4)   ✓
Shape (3,2) + Shape (3,4) → Error          ✗
```

### 🧠 Brain Analogy
Imagine a **hormone** released by one gland (a single value or small tensor) that diffuses through the bloodstream and affects **every cell** that has the right receptor (the larger tensor). The hormone doesn't get copied for each cell — it's broadcast from a single source to reach all targets simultaneously.

Concretely: you have a row vector `[1, 2, 3]` (one set of feature offsets) and you want to add it to a batch of 3 different samples. Instead of manually copying the row 3 times, PyTorch *broadcasts* it — like the same hormonal signal reaching all 3 cells at once.

### ⚙️ Engineer Analogy
Broadcasting is **implicit loop unrolling + memory aliasing**. Instead of allocating a full expanded tensor, PyTorch sets the **stride of a broadcast dimension to 0** — meaning the hardware reads the same memory address repeatedly, once per logical position. No extra memory is allocated.

Rules (aligned from the right):
1. If dimensions differ, prepend 1s to the smaller shape: `(3,)` → `(1, 3)`
2. If a dim is 1, it stretches to match the other: `(1, 4)` → `(3, 4)`
3. If dims are neither equal nor 1 → **error**

This is why `unsqueeze` is so commonly used — to intentionally introduce a size-1 dim so broadcasting can take over.

In [ ]:
# ── Broadcasting in action ────────────────────────────────────────────────
# 🧠 row = one hormonal signal [1, 2, 3] (one set of feature biases)
# 🧠 col = three different cell types [10], [20], [30] (three samples in our batch)
# 🧠 result = every cell type receives every hormonal value → 3×3 response grid

# ⚙️ row shape: (1,3), col shape: (3,1)
#    PyTorch stretches: row → (3,3) with stride=0 on dim-0
#                       col → (3,3) with stride=0 on dim-1
#    No memory allocated for the expanded tensors — purely stride tricks!
row = torch.tensor([[1., 2., 3.]])          # shape (1,3)  ← one "hormone" pattern
col = torch.tensor([[10.], [20.], [30.]])   # shape (3,1)  ← three "cell types"

result = row + col                           # broadcasts to (3,3)
print("Broadcast result shape:", result.shape)
print(result)
# Expected:
# [[11, 12, 13],   ← cell-type 1 (10) receives hormone [1,2,3] → [11,12,13]
#  [21, 22, 23],   ← cell-type 2 (20) receives hormone [1,2,3] → [21,22,23]
#  [31, 32, 33]]   ← cell-type 3 (30) receives hormone [1,2,3] → [31,32,33]

# ── Real-world use: adding a bias vector to a batch of activations ────────
# 🧠 In a neural layer: every sample gets the SAME bias added (broadcast over batch)
# ⚙️ batch shape (32, 10) + bias shape (10,) → bias broadcasts to (32, 10)
batch      = torch.randn(32, 10)   # 32 samples, 10 features each
bias       = torch.randn(10)       # one shared bias per feature
out        = batch + bias          # (32,10) + (10,) → broadcast → (32,10)
print(f"\nBatch + bias shape: {out.shape}")   # still (32, 10)

## 1.6  Moving to GPU

This is one of PyTorch's superpowers — seamlessly move tensors between devices.

### 🧠 Brain Analogy
Think of the **CPU as your prefrontal cortex** — slow, deliberate, sequential thinking. It's great for complex logic, decision-making, and tasks that require one step to complete before the next can begin.

Think of the **GPU as your cerebellum** — fast, massively parallel, handles thousands of simple operations simultaneously. The cerebellum doesn't "think" — it executes learned motor patterns at lightning speed without conscious effort.

When you do matrix multiplications for deep learning (thousands of dot products per forward pass), you want the *cerebellum* doing it, not the prefrontal cortex. Moving a tensor to GPU is like handing the task off from slow conscious thought to fast automatic processing.

### ⚙️ Engineer Analogy
- **CPU** → general-purpose processor: 8–64 cores, each powerful, low-latency, designed for sequential tasks with branching logic
- **GPU** → massively parallel co-processor: 1,000–16,000 simpler cores (CUDA cores/tensor cores), optimised for throughput over latency
- **`.to(device)`** → triggers a **PCIe DMA transfer** from host RAM to device VRAM. This has latency (~1ms), so you minimise transfers by keeping data on GPU as long as possible
- **`device=device` at creation** → allocates directly in VRAM, skipping the transfer entirely
- **`.cpu().numpy()`** → transfers back to host RAM so NumPy/matplotlib can access it (they can't read GPU memory)

> **Engineering rule:** Keep tensors on GPU for the entire forward + backward pass. Only move to CPU at the very end for logging/plotting. Every unnecessary transfer wastes PCIe bandwidth.

In [ ]:
# ── Device setup — define once, use everywhere ───────────────────────────
# 🧠 Deciding which "part of the brain" handles the work — conscious (CPU) or automatic (GPU)
# ⚙️ Best practice: single device variable so you can switch hardware with one line change
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ── Moving an existing tensor to device ──────────────────────────────────
# 🧠 Like transferring a memory from short-term (CPU RAM) to muscle memory (GPU VRAM)
# ⚙️ Triggers a DMA transfer over PCIe bus — has latency cost, so do it once
x = torch.randn(1000, 1000)          # allocated in CPU RAM
x_device = x.to(device)             # move to GPU VRAM (no-op if device='cpu')

print(f"Tensor device: {x_device.device}")

# ── Creating directly on device — skip the transfer ───────────────────────
# 🧠 "Think in GPU memory from the start" — no moving needed
# ⚙️ Allocates in VRAM directly, bypassing PCIe — preferred for large tensors
y_device = torch.randn(1000, 1000, device=device)  # born on GPU

# ── Operations happen where the tensors live ──────────────────────────────
# 🧠 The cerebellum executes 1,000,000 multiply-adds in parallel — takes microseconds
# ⚙️ GPU GEMM kernel: 1000×1000 @ 1000×1000 = 10^9 floating-point ops, ~1ms on modern GPU
result = x_device @ y_device
print(f"Result device: {result.device}, shape: {result.shape}")

# ── Transfer back to CPU only when needed ────────────────────────────────
# 🧠 Bring the result back to conscious awareness for inspection/plotting
# ⚙️ .cpu() → DMA transfer back to host RAM; .numpy() → zero-copy view of that RAM
result_cpu = result.cpu().numpy()
print(f"Back to NumPy: {result_cpu.shape}")

## 1.7  Autograd — The Magic Behind Backpropagation

When `requires_grad=True`, PyTorch records every operation on the tensor and can automatically compute gradients.

### 🧠 Brain Analogy
This is the most important section for understanding how neural networks *learn* — and it maps almost perfectly to how the biological brain learns.

**Hebbian learning** (the biological version): "Neurons that fire together, wire together." When you make a mistake, your brain sends a **dopamine signal** backwards through the network — not from input to output, but from the *result* (reward/error) back to the *cause* (which neurons fired). Synaptic weights that contributed to the error get adjusted.

**Backpropagation** (PyTorch's version) is the same idea, mathematically formalised:
1. **Forward pass** → data flows forward through the network (like a thought forming)
2. **Compute loss** → measure how wrong the output was (like the brain sensing "that was a mistake")
3. **`.backward()`** → error signal flows *backwards* through the computation graph, computing `∂loss/∂weight` for every parameter (like dopamine signalling which synapses caused the error)
4. **Update weights** → adjust parameters in the direction that reduces error (like synaptic plasticity)

PyTorch's autograd **automatically** does step 3 — it tracks every operation and applies the chain rule.

### ⚙️ Engineer Analogy
Autograd is a **symbolic differentiation engine** built on a dynamic computation graph (DAG):
- **`requires_grad=True`** → mark this tensor as a "differentiable parameter" — PyTorch will record every op applied to it
- **Computation graph** → a DAG where nodes are tensors and edges are operations. Built dynamically as you run code (unlike TensorFlow 1.x's static graph)
- **`.backward()`** → triggers reverse-mode automatic differentiation (reverse-mode AD). Traverses the DAG backwards, applying the chain rule at each node: `∂loss/∂x = ∂loss/∂y * ∂y/∂x`
- **`.grad`** → the accumulated gradient tensor (same shape as the parameter)
- **`torch.no_grad()`** → disables the DAG recorder — like turning off the "tape recorder". No graph is built, no gradients stored. Use during inference for speed + memory savings

> **Key engineering principle:** The chain rule is just repeated multiplication of local derivatives. Autograd does this multiplication for you, through potentially thousands of nested operations, without you writing a single line of derivative math.

In [ ]:
# ── Simple gradient example ──────────────────────────────────────────────
# 🧠 x = a single synapse's strength (a learnable parameter)
# 🧠 f = the brain's total "cost" for making this decision
# 🧠 f.backward() = dopamine floods back: "how much did this synapse cause the error?"
# ⚙️ PyTorch records: f = x**3 + 2*x**2 + 5 as a computation graph
#    .backward() applies chain rule: df/dx = 3x² + 4x

x = torch.tensor(3.0, requires_grad=True)  # x=3, flagged as differentiable

# f(x) = x^3 + 2x^2 + 5  — a simple "loss function" in one variable
f = x**3 + 2*x**2 + 5
print(f"f(3)  = {f.item():.1f}")           # f(3) = 27 + 18 + 5 = 50

# ── Backward pass — the "dopamine signal" flows back ─────────────────────
# 🧠 "Which direction should I move x to reduce f?"
# ⚙️ Traverses the computation DAG backwards; at each node applies local derivative
f.backward()
# df/dx at x=3: 3*(3²) + 4*(3) = 27 + 12 = 39
print(f"df/dx = {x.grad.item():.1f}   # analytical: 3x²+4x = 27+12 = 39")
# Gradient > 0 means: increasing x increases f — so to DECREASE f, move x left (subtract)

# ── torch.no_grad() — turn off the tape recorder ────────────────────────
# 🧠 Like doing something on "autopilot" — you're not learning from this, just acting
# ⚙️ Disables autograd DAG construction: ~30% faster, uses less memory
#    Essential during: model inference, parameter updates (otherwise PyTorch tracks
#    the update itself and tries to backprop through it!)
with torch.no_grad():
    y = x * 2
print(f"\nWith no_grad, requires_grad: {y.requires_grad}")   # False

In [ ]:
# ── Gradient w.r.t. parameters — the neural net analogy ─────────────────
# 🧠 This IS a single neuron layer:
#    W = synaptic weights (how strongly each input neuron connects to each output neuron)
#    b = bias (the baseline firing threshold even with zero input)
#    x_in = the sensory signal coming in (3 input features → like 3 receptor types)
#    y_pred = the output activations (what this layer "thinks" — 2 output neurons)
#    loss = how wrong the prediction was (the dopamine error signal)
#
# ⚙️ W shape (2,3): a linear transformation from 3D input space to 2D output space.
#    Mathematically: y = Wx + b  — this IS the core of nn.Linear (minus activation fn)

torch.manual_seed(42)
W = torch.randn(2, 3, requires_grad=True)   # weight matrix: 2 outputs ← 3 inputs
b = torch.zeros(2,   requires_grad=True)    # bias: one value per output neuron

x_in = torch.randn(3)                       # one sample with 3 input features
y_pred = W @ x_in + b                       # linear layer forward pass → shape (2,)
                                             # 🧠 The 2 output neurons receive & combine
                                             #    the 3 input signals, weighted by W

loss   = (y_pred ** 2).sum()                # simple loss: sum of squared activations
                                             # 🧠 "How loudly are the output neurons firing?
                                             #    We want them quiet (close to 0)"

# ── Backward: error signal propagates back through W and b ────────────────
# 🧠 Dopamine signal: "W[i,j] — you contributed this much to the error.
#    Your synaptic strength should change by W.grad[i,j]"
# ⚙️ Chain rule: ∂loss/∂W = ∂loss/∂y_pred * ∂y_pred/∂W = 2*y_pred * x_in (outer product)
loss.backward()

print("W.grad:\n", W.grad)  # shape (2,3): gradient for every synapse
print("b.grad :", b.grad)   # shape (2,):  gradient for every bias neuron
# Gradient sign: positive → increasing that weight increases loss → should DECREASE it
# Gradient magnitude: how SENSITIVE the loss is to that weight

## 1.8  Real-World Mini Project: Linear Regression from Scratch

**Use Case:** Predict house prices from square footage.  
No `nn.Module` yet — we build everything manually to understand the training loop.

**Data:** Synthetic (mimics what you'd get from a Kaggle regression dataset)

---

### 🧠 Brain Analogy — Learning by Experience
Imagine a child learning to estimate room sizes. They walk into rooms, guess the size, get corrected, and gradually improve. This is *exactly* the training loop:

1. **See a house** (forward pass: compute a prediction)
2. **Estimate its price** (y_pred = W * sqft + b)
3. **Get corrected** ("actual price was $350k, you said $280k — you're off by $70k")
4. **Compute the error** (loss = MSE between prediction and truth)
5. **Adjust your mental model** (backprop: compute gradients → update W and b)
6. **Repeat with the next house** (next epoch iteration)

After hundreds of houses, the child's internal model (W, b) converges to something close to the true relationship. The brain has learned `price ≈ 300 * sqft + 50,000`.

### ⚙️ Engineer Analogy — PID Controller / System Identification
This training loop is mathematically identical to **gradient descent system identification**:
- **W, b** → the unknown parameters of a linear system `y = Wx + b` you're trying to identify
- **Loss (MSE)** → the squared error between your model's output and the measured system output
- **Gradient** → the direction in parameter space that most increases the error — so you step the *opposite* direction
- **Learning rate `lr`** → the step size; too large → oscillates and diverges (unstable control); too small → converges slowly
- **Normalisation** → shifting and scaling input/output to `~N(0,1)` prevents gradients from exploding or vanishing — equivalent to **pre-conditioning** a linear system for better numerical stability

> **Why zero the gradients?** PyTorch *accumulates* gradients by default (adds to `.grad` rather than replacing). This is useful for certain advanced techniques (gradient accumulation over mini-batches), but for a standard loop you must zero them each step or you're adding stale gradients from the previous iteration — like a PID controller that never resets its integral term.

In [ ]:
# ── Generate synthetic house price data ─────────────────────────────────
# 🧠 Like a teacher preparing 200 flashcards: each card shows a house size,
#    and the "answer" on the back is the true price (with a bit of real-world noise)
# ⚙️ We define the ground-truth relationship: price = 300*sqft + 50000 + noise
#    This is the "oracle" — what our model is trying to learn

torch.manual_seed(0)
np.random.seed(0)

# True relationship: price = 300 * sqft + 50000 + noise
sqft  = torch.linspace(500, 3500, 200).reshape(-1, 1)   # shape (200,1): 200 houses
noise = torch.randn(200, 1) * 20000                      # real-world noise (~$20k std)
price = 300 * sqft + 50000 + noise                       # shape (200,1): true prices

# ── Normalise — pre-condition the system for stable gradients ─────────────
# 🧠 Like converting everything to a common scale before comparing:
#    "sqft" ranges from 500–3500; "price" ranges from 200k–1.1M
#    Hard to learn across such different scales — normalise so both are in [-2, +2]
# ⚙️ Z-score normalisation: x_norm = (x - mean) / std
#    → both inputs and outputs have mean≈0, std≈1
#    → gradients are well-conditioned (similar magnitudes), converges faster
sqft_norm  = (sqft  - sqft.mean())  / sqft.std()
price_norm = (price - price.mean()) / price.std()

# ── Visualise the raw data ────────────────────────────────────────────────
# 🧠 Like the teacher showing you all 200 flashcards at once — see the pattern?
# ⚙️ Each dot = one training example (sqft, price). The signal-to-noise ratio
#    is high here, so linear regression should work well.
plt.figure(figsize=(7, 4))
plt.scatter(sqft.numpy(), price.numpy(), alpha=0.5, s=20)
plt.xlabel("Square Footage")
plt.ylabel("Price ($)")
plt.title("Synthetic House Price Data (True: price = 300·sqft + 50k + noise)")
plt.tight_layout()
plt.show()

In [ ]:
# ── Linear Regression Training Loop (pure tensor math) ──────────────────
#
# 🧠 THE LEARNING LOOP — 5 steps that mirror how the brain learns:
#   Step 1: PERCEIVE   (forward pass — form a prediction)
#   Step 2: EVALUATE   (compute loss — how wrong was that prediction?)
#   Step 3: REFLECT    (backward pass — which weights caused the error?)
#   Step 4: ADJUST     (gradient descent — update weights to reduce error)
#   Step 5: RESET      (zero gradients — clear the "working memory" for next round)
#
# ⚙️ This loop implements Gradient Descent optimisation:
#   θ ← θ - lr * ∂L/∂θ    (where θ = parameters, L = loss, lr = step size)
#   It's equivalent to the Widrow-Hoff LMS (Least Mean Squares) update rule
#   used in adaptive filters and signal processing.

W = torch.randn(1, 1, requires_grad=True)   # weight: guess of slope     (initially random)
b = torch.zeros(1,    requires_grad=True)   # bias:   guess of intercept (initially 0)

lr     = 0.01   # step size — like the "learning rate" of a student (too high → panic, too low → slow)
epochs = 200    # number of times we show ALL 200 houses to the model
losses = []

for epoch in range(epochs):
    # ── Step 1: FORWARD PASS — form a prediction ─────────────────────────
    # 🧠 "Given this house size, what price do I think it is?" (W·sqft + b)
    # ⚙️ Matrix multiply: (200,1) @ (1,1) + (1,) → (200,1) — one prediction per house
    y_pred = sqft_norm @ W + b

    # ── Step 2: COMPUTE LOSS — measure the error ─────────────────────────
    # 🧠 "How far off was I? Across all 200 houses?"
    # ⚙️ MSE = mean( (ŷ - y)² ) — the squared error penalises large mistakes more
    loss = ((y_pred - price_norm) ** 2).mean()

    # ── Step 3: BACKWARD PASS — compute gradients ────────────────────────
    # 🧠 "Which way should W and b move to reduce my mistakes?"
    # ⚙️ Autograd traverses DAG backwards; populates W.grad and b.grad
    loss.backward()

    # ── Step 4: UPDATE PARAMETERS — gradient descent step ────────────────
    # 🧠 "Adjust my mental model slightly in the right direction"
    # ⚙️ MUST use no_grad: we don't want PyTorch to track the update itself
    #    (otherwise it would try to backprop through the update — circular!)
    with torch.no_grad():
        W -= lr * W.grad    # move W opposite to gradient (downhill on loss surface)
        b -= lr * b.grad    # same for bias

    # ── Step 5: ZERO GRADIENTS — clear working memory ────────────────────
    # 🧠 "Forget the gradient from this round — don't let it contaminate the next"
    # ⚙️ PyTorch accumulates grads by default (.grad += new_grad each backward())
    #    We MUST zero before the next forward pass, or gradients compound incorrectly
    W.grad.zero_()
    b.grad.zero_()

    losses.append(loss.item())

print(f"Final loss: {losses[-1]:.4f}")
print(f"Learned W: {W.item():.4f}, b: {b.item():.4f}")
print(f"(In normalised space — both should be close to the true normalised slope & intercept)")

In [ ]:
# ── Visualise training ────────────────────────────────────────────────────
# 🧠 Left plot: the "learning curve" — like charting a student's improvement over time.
#    A healthy brain shows rapid early improvement (steep drop) then plateaus.
#    If it never drops → the model isn't learning (lr too small or wrong architecture).
#    If it drops then spikes → the model is unstable (lr too large).
#
# 🧠 Right plot: the "fitted model" — has the brain internalized the true relationship?
#    Red line = the model's belief about how sqft → price after 200 training rounds.
#    Blue dots = the ground truth examples it learned from.
#
# ⚙️ Left: Loss curve should show exponential-ish decay → log-linear in log-space.
#    This is characteristic of gradient descent on a convex loss surface (MSE is convex!).
# ⚙️ Right: A straight-line fit on normalised data. W ≈ 1.0 and b ≈ 0.0 means
#    the model has recovered the true linear relationship in normalised space.

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# ── Loss curve (learning progress) ───────────────────────────────────────
ax1.plot(losses, color='steelblue')
ax1.set_xlabel("Epoch (training round)")
ax1.set_ylabel("MSE Loss")
ax1.set_title("Training Loss Curve\n🧠 Student improving | ⚙️ Gradient descent on convex surface")
ax1.axhline(y=losses[-1], color='red', linestyle='--', alpha=0.5, label=f'Final: {losses[-1]:.4f}')
ax1.legend()

# ── Regression fit ────────────────────────────────────────────────────────
with torch.no_grad():          # inference only — no need to track gradients
    pred = sqft_norm @ W + b   # apply learned W, b to all 200 normalised inputs

ax2.scatter(sqft_norm.numpy(), price_norm.numpy(), alpha=0.4, s=20, label="Training data")
ax2.plot(sqft_norm.numpy(), pred.numpy(), color='red', lw=2,
         label=f"Learned fit  W={W.item():.3f}, b={b.item():.3f}")
ax2.set_xlabel("sqft (normalised)")
ax2.set_ylabel("price (normalised)")
ax2.set_title("Linear Regression Fit\n🧠 Learned belief | ⚙️ Identified system parameters")
ax2.legend()

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print(f"  Learned W={W.item():.4f} in normalised space")
print(f"  Since data is z-scored, W≈1.0 confirms the model learned the dominant linear trend.")
print(f"  b≈0.0 confirms the normalised intercept (mean is already removed by z-scoring).")

## 1.9  Quick Reference — Tensor Cheatsheet

| Operation | Code | Notes |
|-----------|------|-------|
| Create | `torch.tensor(data)` | from Python list/ndarray |
| Zeros | `torch.zeros(r, c)` | |
| Random | `torch.randn(r, c)` | N(0,1) |
| Shape | `x.shape` or `x.size()` | |
| Reshape | `x.reshape(a, b)` | |
| Flatten | `x.flatten()` | |
| To device | `x.to(device)` | |
| To numpy | `x.cpu().numpy()` | |
| MatMul | `a @ b` | |
| Grad | `x.backward()` | |
| No grad | `torch.no_grad()` | inference only |

## Exercises

1. Create a tensor of shape `(5, 5)` with random values and find the row with the highest mean.
2. Implement the gradient of `f(x) = sin(x²)` at `x = π/4` using autograd. Verify with calculus.
3. Modify the house price model to use **batch gradient descent** (update every N samples).

---
**Next →** [Module 02: Your First Neural Network with nn.Module](./Module_02_First_Neural_Network.ipynb)